# BUILDING THE jobs_execution_info.csv FILE.
The file composes together the information about each job (in jobs_data.csv) with the measurements from the sensors (in sensors_measures.csv)

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import display
#pd.reset_option('display.max_rows', silent=True)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.options.display.float_format = None


sdf = pd.read_csv('sensors_measures.csv', index_col=None,header=0, dtype={'notes': "string", 'delta_e': np.float64})
sdf.head(20)
#sdf.set_index(['jobid', 'measure_ts', 'nodename']).index.is_unique


,jobid,measure_ts,nodename,sys_power,cpu_power,mem_power,sys_util,cpu_util,mem_util,io_util,amb_temp,cpu1_temp,cpu2_temp,exh_temp,sysairflow,fan1a,fan1b,fan2a,fan2b,fan3a,fan3b,fan4a,fan4b,fan5a,fan5b,node_energy,notes,delta_e
0,944853,1704495940,187,130,100,10,85,97,0,0,21,55,53,31,14,6400,5504,8320,8064,8448,8064,8320,8064,6400,5504,394.84890,<NA>,0.00004
1,944853,1704495940,328,80,50,5,54,97,0,0,18,65,55,41,20,6272,6272,8320,8704,8320,8704,8320,8704,6144,6272,264.67395,<NA>,0.00005
2,944853,1704495940,132,120,90,9,81,98,40,0,21,75,54,44,23,6272,6272,8320,8704,8320,8704,8320,8704,6144,6272,400.03630,<NA>,0.00007
3,944853,1704495938,187,130,100,9,85,97,0,0,21,61,53,31,23,6400,5504,8320,8064,8448,8064,8320,8064,6400,5504,394.84886,<NA>,0.00008
4,944853,1704495938,328,80,50,5,98,97,0,0,18,65,55,41,20,6272,6272,8320,8704,8320,8704,8320,8704,6144,6272,264.67390,<NA>,0.00005
5,944853,1704495938,132,140,100,9,81,98,40,0,21,75,64,44,23,6272,6272,8320,8704,8320,8704,8320,8704,6144,6272,400.03623,<NA>,0.00007
6,944853,1704495937,187,130,100,9,85,97,0,0,21,61,62,31,23,6400,5504,8320,8064,8448,8064,8320,8064,6400,5504,394.84878,<NA>,0.00007
7,944853,1704495936,132,140,100,9,80,98,38,0,21,75,64,44,23,6272,6272,8320,8704,8320,8704,8320,8704,6144,6272,400.03616,<NA>,0.00008
8,944853,1704495936,328,80,50,5,98,97,0,0,18,65,55,41,20,6272,6272,8320,8704,8320,8704,8320,8704,6144,6272,264.67385,<NA>,0.00004
9,944853,1704495935,187,130,100,9,98,97,0,0,21,61,62,41,23,6400,5504,8320,8064,8448,8064,8320,8064,6400,5504,394.84871,<NA>,0.00008


In [2]:
##ONLY FOR TESTING
#sdf = sdf.loc[sdf['jobid'].isin([909611 ])] #944853
#energy=0
#for index, row in sdf.iterrows():
#    energy+=row['delta_e']
#print(energy)    
#display(sdf)

In [3]:
jdf = pd.read_csv('jobs_data.csv', index_col=None,header=0)
jdf.head()


,jobid,jobname,start_ts,end_ts,nodes,condition_n,algorithm,fault_tolerance,fault_happened,computing_proc,total_proc,matrix_size,precision,balanced,repetition,req_whole_time,number_of_nodes,node1,node2,node3,node4,node5,node6,node7,node8,node9,node10,node11,node12,node13,node14,node15,node16,IMe_fault_level,IMe_fault_rank,SPK_checkpoint_iteration,blocking_factor,NxSOCK,R,error_code,runtime,init_and_call_runtime,call_runtime
0,891576,SPKFT2_nf2_cp576_tnp578_ms42240_double_lbn_r1_...,1701589448,1701589519,48*cresco6x051:48*cresco6x114:48*cresco6x147:4...,1,SPK,2,2,576,578,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99,71,47,32
1,891577,IMeCOFT4_nf0_cp576_tnp672_ms42240_double_lbn_r...,1701589543,1701590227,48*cresco6x214:48*cresco6x113:48*cresco6x230:4...,1,IMeCO,4,0,576,672,42240,double,n,1,15,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,-,-,-,1.0,0,685,659,652
2,891578,SPKFT4_nf0_cp576_tnp580_ms42240_double_lbn_r1_...,1701590250,1701590319,48*cresco6x299:48*cresco6x246:48*cresco6x221:4...,1,SPK,4,0,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,0,69,47,32
3,891579,IMeCOFT4_nf4_cp576_tnp672_ms42240_double_lbn_r...,1701590345,1701590957,48*cresco6x229:48*cresco6x303:48*cresco6x113:4...,1,IMeCO,4,4,576,672,42240,double,n,1,15,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,-,-,-,1.0,0,612,588,581
4,891580,SPKFT4_nf4_cp576_tnp580_ms42240_double_lbn_r1_...,1701590976,1701591048,48*cresco6x188:48*cresco6x062:48*cresco6x081:4...,1,SPK,4,4,576,580,42240,double,n,1,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,21120,22,-,1.0,-99,72,47,32


In [4]:
energy_df = sdf.filter(['jobid','delta_e'], axis=1).groupby('jobid').sum()
energy_df= energy_df.rename(columns={"delta_e": "energy_from_delta_e"})
energy_df.head()


,energy_from_delta_e
jobid,
879962,0.03901
879963,0.00398
879964,0.03851
879965,0.00423
879966,0.03864


In [5]:
energy2_df = sdf.filter(['jobid','measure_ts','nodename','node_energy'], axis=1)
energy_min_df = energy2_df.groupby(['jobid','nodename']).min('measure_ts').rename(columns={"node_energy": "starting_node_energy"})#.drop(columns=['measure_ts'])
energy_max_df = energy2_df.groupby(['jobid','nodename']).max('measure_ts').rename(columns={"node_energy": "ending_node_energy"})#.drop(columns=['measure_ts'])
display(energy_min_df)
display(energy_max_df)

energy_min_df=energy_min_df.drop(columns=['measure_ts'])
energy_max_df=energy_max_df.drop(columns=['measure_ts'])


energy3_df = pd.concat([energy_min_df, energy_max_df], axis=1, join="inner")
energy3_df['energy_from_subtraction'] = energy3_df['ending_node_energy'] - energy3_df['starting_node_energy']
display(energy3_df)

energy4_df = energy3_df.filter(['jobid','energy_from_subtraction'], axis=1).groupby('jobid').sum()
display(energy4_df)


measure_ts  starting_node_energy
jobid  nodename                                  
879962 114       1700602023            1858.41351
       184       1700602023             105.33058
       186       1700602023             114.13925
879963 149       1700602228             108.90230
       226       1700602227             116.11544
...                     ...                   ...
944861 188       1704502090             390.96254
945323 126       1704524924             417.70439
       278       1704524924             379.32344
       292       1704524924             394.98841
       304       1704524924             343.83540

[55506 rows x 2 columns]

measure_ts  ending_node_energy
jobid  nodename                                
879962 114       1700602212          1858.43085
       184       1700602212           105.33758
       186       1700602212           114.15392
879963 149       1700602254           108.90397
       226       1700602254           116.11692
...                     ...                 ...
944861 188       1704502968           391.03505
945323 126       1704525803           417.77221
       278       1704525805           379.40068
       292       1704525804           395.06123
       304       1704525804           343.91125

[55506 rows x 2 columns]

starting_node_energy  ending_node_energy  \
jobid  nodename                                             
879962 114                 1858.41351          1858.43085   
       184                  105.33058           105.33758   
       186                  114.13925           114.15392   
879963 149                  108.90230           108.90397   
       226                  116.11544           116.11692   
...                               ...                 ...   
944861 188                  390.96254           391.03505   
945323 126                  417.70439           417.77221   
       278                  379.32344           379.40068   
       292                  394.98841           395.06123   
       304                  343.83540           343.91125   

                 energy_from_subtraction  
jobid  nodename                           
879962 114                       0.01734  
       184                       0.00700  
       186                       0.01467  
879963 149                       0.00167  
       226                       0.00148  
...                                  ...  
944861 188                       0.07251  
945323 126                       0.06782  
       278                       0.07724  
       292                       0.07282  
       304                       0.07585  

[55506 rows x 3 columns]

,energy_from_subtraction
jobid,
879962,0.03901
879963,0.00398
879964,0.03851
879965,0.00423
879966,0.03864
...,...
944858,0.11754
944859,0.21773
944860,0.11577


In [6]:
power_series_df = sdf.filter(['jobid','measure_ts','sys_power'], axis=1).groupby(['jobid', 'measure_ts']).agg({'sys_power': 'sum'})
power_series_df.head()

sys_power
jobid  measure_ts           
879962 1700602023        380
       1700602024        340
       1700602025        360
       1700602026        350
       1700602027        360

In [7]:
power_df = power_series_df.groupby('jobid').agg({'sys_power': ['mean', 'max']})
power_df.columns = power_df.columns.get_level_values(1)
power_df= power_df.rename(columns={"mean": "mean_sys_power", "max": "max_sys_power"})

power_df.head()

,mean_sys_power,max_sys_power
jobid,,
879962,740.631579,860
879963,523.928571,800
879964,754.324324,880
879965,587.407407,810
879966,754.702703,880


In [8]:
info_df=energy_df.join(energy4_df, on='jobid').join(power_df, on='jobid')
info_df['energy_equal']= (np.isclose(info_df['energy_from_delta_e'],info_df['energy_from_subtraction']))
#print(str(len(info_df.loc[info_df['energy_equal']==False])))
#info_df.loc[info_df['energy_equal']==False].head()
#info_df=info_df.loc[info_df['energy_equal']==True]
info_df.head()

,energy_from_delta_e,energy_from_subtraction,mean_sys_power,max_sys_power,energy_equal
jobid,,,,,
879962,0.03901,0.03901,740.631579,860,True
879963,0.00398,0.00398,523.928571,800,True
879964,0.03851,0.03851,754.324324,880,True
879965,0.00423,0.00423,587.407407,810,True
879966,0.03864,0.03864,754.702703,880,True


In [9]:
filtered_jdf_df = jdf.filter(['jobid','error_code','runtime','matrix_size', 'precision','balanced','number_of_nodes'], axis=1)
filtered_jdf_df.head()
info_df=info_df.join(filtered_jdf_df.set_index('jobid'), on='jobid')
info_df

,energy_from_delta_e,energy_from_subtraction,mean_sys_power,max_sys_power,energy_equal,error_code,runtime,matrix_size,precision,balanced,number_of_nodes
jobid,,,,,,,,,,,
879962,0.03901,0.03901,740.631579,860,True,0.0,184.0,21120.0,single,n,3.0
879963,0.00398,0.00398,523.928571,800,True,0.0,21.0,21120.0,single,n,3.0
879964,0.03851,0.03851,754.324324,880,True,0.0,178.0,21120.0,single,n,3.0
879965,0.00423,0.00423,587.407407,810,True,0.0,21.0,21120.0,single,n,3.0
879966,0.03864,0.03864,754.702703,880,True,0.0,178.0,21120.0,single,n,3.0
...,...,...,...,...,...,...,...,...,...,...,...
944858,0.11754,0.11754,569.576060,1080,True,0.0,447.0,31680.0,single,n,3.0
944859,0.21773,0.21773,562.274563,970,True,0.0,872.0,31680.0,double,n,3.0
944860,0.11577,0.11577,560.075377,1090,True,0.0,443.0,31680.0,single,y,3.0


In [10]:
info_df.to_csv('jobs_info.csv', float_format='%f',index=True)
print("jobs_info.csv DONE!")

jobs_info.csv DONE!
